In [14]:
import sys
import os
import collections
from collections import defaultdict
import numpy as np
import pandas as pd
from scipy import stats
import re

df = pd.read_csv('Sample_data.csv')
df

,Requirements,Risk,Documentation,Stakeholder Involvement,PMMname
0,2,9,8,9,Waterfall
1,3,9,7,8,Waterfall
2,2,9,7,9,Waterfall
3,3,8,8,9,Waterfall
4,2,9,8,9,Waterfall
...,...,...,...,...,...
955,5,3,8,3,Kanban
956,5,3,8,3,Kanban
957,5,2,8,4,Kanban
958,5,3,8,3,Kanban


In [2]:
import math
from sklearn import neighbors, datasets
from numpy.random import permutation
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support

In [3]:
#similar_univs = pandas.read_csv('similar_universities.csv')
random_indices = permutation(df.index)
test_cutoff = math.floor(len(df)/5)
print(test_cutoff)
test = df.loc[random_indices[1:test_cutoff]]
train = df.loc[random_indices[test_cutoff:]]
train_output_data = train['PMMname']
print("train Output data", train_output_data)
train_input_data = train
train_input_data = train_input_data.drop('PMMname',1)
print("train input data", train_input_data)
test_output_data = test['PMMname']
print("test Output data", test_output_data)
test_input_data = test
test_input_data = test_input_data.drop('PMMname',1)
print("test input data", test_input_data)

192
train Output data 698       Scrum 
496      Spiral 
15       Spiral 
309       Scrum 
656       Scrum 
         ...    
606       Scrum 
361       Scrum 
87        Kanban
957       Kanban
197    Waterfall
Name: PMMname, Length: 768, dtype: object
train input data      Requirements   Risk  Documentation  Stakeholder Involvement
698              9     3              3                        3
496              2     8              7                        8
15               3     9              7                        7
309             10     2              3                        4
656             10     2              4                        3
..             ...   ...            ...                      ...
606             10     2              2                        3
361             10     2              3                        2
87               6     4              8                        3
957              5     2              8                        4
197              

/var/folders/b5/rpp5gn7169g6990v79kvffzc0000gp/T/ipykernel_2458/910898578.py:10: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  train_input_data = train_input_data.drop('PMMname',1)
/var/folders/b5/rpp5gn7169g6990v79kvffzc0000gp/T/ipykernel_2458/910898578.py:15: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  test_input_data = test_input_data.drop('PMMname',1)


In [4]:
def euclideanDistance(data1, data2, length):
    distance = 0
    
    #calculating distance
    for x in range(length):
        distance += np.square(data1[x] - data2[x])
    return np.sqrt(distance)

def knn(trainingSet, testInstance, k):
    print(k)
    distances = {}
    sort = {}
    length = testInstance.shape[1]

    for x in range(len(trainingSet)):

        dist = euclideanDistance(testInstance, trainingSet.iloc[x], length)
        
        #getting closest neighbours to the instance
        distances[x] = dist[0]

    #adding the 2nd element in the list to the sort function (neighbour)
    sorted_d = sorted(distances.items(), key=lambda x: x[1])

 
    neighbors = []

    for x in range(k):
        neighbors.append(sorted_d[x][0])

    classVotes = {}

    for x in range(len(neighbors)):
        response = trainingSet.iloc[neighbors[x]][-1]
        if response in classVotes:
            classVotes[response] += 1
        else:
            classVotes[response] = 1

    sortedVotes = sorted(classVotes.items(), key=lambda x: x[1], reverse=True)
 
    return(sortedVotes, neighbors)

In [15]:
#testing the algorithm 
testSet = test
test = pd.DataFrame(testSet)
test.shape
test

,0,1,2,3
0,9,3,3,3


In [13]:
#setting k
k = 250

result,neigh= knn(df, test, k)

list1 = []
list2 = []
for i in result:
    list1.append(i[0])
    list2.append(i[1])
for i in list1:
    print(i)

250
Scrum 


In [12]:
df = pd.read_csv('Sample_data.csv')
df.shape


def sdlc_weighted_rating(sdlc, c, m):
    nv = sdlc['tot_count']
    av = sdlc['mean']
    rating = (nv/(nv+m) * av) + (m/(m+nv) * c)
    return rating

In [ ]:
df['Total']= df.iloc[:, 1:5].sum(axis=1)
print (df)

In [ ]:
df['PMMname'].value_counts()
rat = df.groupby('PMMname').agg({'Total': ['mean']})
rat['Total count'] = df['PMMname'].value_counts()
rat['mean'] = df.groupby('PMMname').agg({'Total': ['mean']})
#rat.rename(columns={ rat.columns[0]: "mean" }, inplace = True)
rat= rat.iloc[:, 1:]

rat



In [ ]:
df.set_index('PMMname')
meta = pd.merge(rat,df,on='PMMname')
meta.rename(columns={ meta.columns[2]: "mean" }, inplace = True)
meta.rename(columns={ meta.columns[1]: "tot_count" }, inplace = True)
meta.dtypes


In [ ]:
def get_recemmendations(sdlcs, req=0, risk=0, doc =0, stak=0, recomm_count=4, percent=0.7):
    if req != 0:
        sdlcs = sdlcs[sdlcs['Requirements ']>=req]
    if risk != 0:
        sdlcs = sdlcs[sdlcs['Risk']>=risk]   
    if doc != 0:
        sdlcs = sdlcs[sdlcs['Documentation']>=doc]
    if stak != 0:
        sdlcs = sdlcs[sdlcs['Stakeholder Involvement']>=stak]  

    all_sdlcs_avg = sdlcs['mean'].mean()
    min_req_vote = sdlcs['tot_count'].quantile(percent)
    result = sdlcs.copy().loc[sdlcs['tot_count'] >= min_req_vote]

    result['rating'] = result.apply(sdlc_weighted_rating, c=all_sdlcs_avg, m=min_req_vote, axis=1)
    result = result.sort_values('rating',ascending = False)[:recomm_count]

    #result['PMMname'] = result['PMMname'].apply(lambda x: x.PMMname())
    
    return result

In [ ]:
#@title Enter you preferences
#@markdown in the form below, enter your preferences (leave them as default if it doesn't matter)

requirements = 1#@param {type:"integer"}
risks =  9#@param {type:"integer"}
docs =  8#@param {type:"integer"}
stake =7  #@param {type:"integer"}
recommendation_count = 1 #@param {type:"integer"}

In [ ]:
get_recemmendations(meta, req=requirements,risk=risks,
                    doc=docs, stak = stake, recomm_count=1)